In [ ]:
!pip install ultralytics
!python -m pip install --extra-index-url https://google-coral.github.io/py-repo/ pycoral
!pip install tensorflow==2.13.1

In [1]:
import numpy as np
from PIL import Image
import tensorflow as tf
from PIL import Image, ImageDraw


In [2]:
from ultralytics import YOLO
model = YOLO('yolov8m.pt')
results = model.export(format='tflite')

Ultralytics YOLOv8.1.15 🚀 Python-3.11.4 torch-2.2.0+cpu CPU (Intel Core(TM) i7-1065G7 1.30GHz)
YOLOv8m summary (fused): 218 layers, 25886080 parameters, 0 gradients, 78.9 GFLOPs

PyTorch: starting from 'yolov8m.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (49.7 MB)

TensorFlow SavedModel: starting export with tensorflow 2.15.0...
WARNING ⚠️ tensorflow<=2.13.1 is required, but tensorflow==2.15.0 is currently installed https://github.com/ultralytics/ultralytics/issues/5161

ONNX: starting export with onnx 1.15.0 opset 17...
ONNX: simplifying with onnxsim 0.4.35...
ONNX: export success ✅ 13.0s, saved as 'yolov8m.onnx' (99.0 MB)
TensorFlow SavedModel: running 'onnx2tf -i "yolov8m.onnx" -o "yolov8m_saved_model" -nuo --non_verbose'
TensorFlow SavedModel: export success ✅ 75.9s, saved as 'yolov8m_saved_model' (247.7 MB)

TensorFlow Lite: starting export with tensorflow 2.15.0...
TensorFlow Lite: export success ✅ 0.0s, saved as 'yolov8m_saved_model\yolov8m_float

In [1]:
import tkinter as tk
import cv2
import numpy as np
from PIL import Image, ImageTk
from tkinter import filedialog
import tensorflow as tf
import tflite_runtime.interpreter as tflite
from tflite_runtime.interpreter import load_delegate
import json
import os

# Load the TensorFlow Lite model with Edge TPU delegate
model_path = 'yolov8m_saved_model/yolov8m_float32.tflite'
interpreter = tflite.Interpreter(model_path=model_path, experimental_delegates=[load_delegate('edgetpu.dll')])
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
input_shape = input_details[0]['shape'][1:]
print("Expected Input Dimensions:", input_details[0]['shape'][1:])

# Print input and output details
print("Input Details:")
print(input_details)
print("Output Details:")
print(output_details)

# Threshold Setting
threshold = 0.03

# Load the class labels from the JSON file
with open('C:\object-detection-coral\label_files\labels_coco.json', 'r') as json_file:
    class_labels = json.load(json_file)

# Create a tkinter window
root = tk.Tk()
root.title("Object Detection")






# save_image_with_boxes
output_directory = "output_images"
os.makedirs(output_directory, exist_ok=True)
def save_image_with_boxes(image, file_path):
    output_path = os.path.join(output_directory, os.path.basename(file_path))
    cv2.imwrite(output_path, image)
    print(f"Image enregistrée avec succès : {output_path}")



# Preprocessing function using TensorFlow
def preprocess(image_path, input_shape):
    # Read image file
    raw = tf.io.read_file(image_path)
    # Decode image
    image = tf.image.decode_image(raw, channels=3)
    # Resize image
    image_resized = tf.image.resize(image, (input_shape[0], input_shape[1]))
    # Convert image to float32
    image_float32 = tf.cast(image_resized, tf.float32)
    # Normalize image
    image_normalized = image_float32 / 255.0
    # Add batch dimension
    image_with_batch = tf.expand_dims(image_normalized, 0)

    return image_with_batch

# Function to open a file dialog and select an image
def open_image():
    file_path = filedialog.askopenfilename()
    if file_path:
        # Preprocess the image
        image_np = preprocess(file_path, input_shape)

        # Perform inference
        interpreter.set_tensor(input_details[0]['index'], image_np)
        interpreter.invoke()
        output = interpreter.get_tensor(output_details[0]['index'])
        output = output[0]
        print(output.shape)

        # Read the image with OpenCV to draw bounding boxes
        image = cv2.imread(file_path)
        orig_h, orig_w = image.shape[:2]

        for i in range(output.shape[1]):
            detection = output[:, i]
            x_center, y_center, width, height = detection[:4]
            x1 = max(0, int((x_center - width / 2) * orig_w))
            y1 = max(0, int((y_center - height / 2) * orig_h))
            x2 = min(orig_w - 1, int((x_center + width / 2) * orig_w))
            y2 = min(orig_h - 1, int((y_center + height / 2) * orig_h))
            score = np.max(detection[4:])
            cls = np.argmax(detection[4:])
            if score >= threshold:
                class_name = class_labels[str(cls)]  # Obtain the class name using the class index
                cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), thickness=2)  # Draw bounding box
                cv2.putText(image, f"{class_name}: {score:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Convert the image to RGB format for display with Tkinter
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image_pil = Image.fromarray(image_rgb)
        tk_image = ImageTk.PhotoImage(image_pil)

        # Update the image label with the new image
        image_label.config(image=tk_image)
        image_label.image = tk_image
        save_image_with_boxes(image, file_path)

# Button to select an image
select_button = tk.Button(root, text="Select Image", command=open_image)
select_button.pack()

# Create a label to initially display the image
image_label = tk.Label(root)
image_label.pack()

# Run the tkinter event loop
root.mainloop()
#s7i7a


Expected Input Dimensions: [640 640   3]
Input Details:
[{'name': 'inputs_0', 'index': 0, 'shape': array([  1, 640, 640,   3]), 'shape_signature': array([  1, 640, 640,   3]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Output Details:
[{'name': 'Identity', 'index': 514, 'shape': array([   1,   84, 8400]), 'shape_signature': array([   1,   84, 8400]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
(84, 8400)
Image enregistrée avec succès : output_images\dog-cat.png
